# 驾安配订单数据
人保订单，德系、日系、美系、韩系车100+品牌，8000 OE，主要是保险杠和大灯

In [ ]:
# 初始化
%matplotlib inline
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
DATA_PATH = 'orders20200909.csv'
PARAM_TIMES = 1.0
MIN_PARAM_TIMES = 0.4
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
# 加载数据
def load_data(csv_path=DATA_PATH):
    return pd.read_csv(csv_path)

orders = load_data(DATA_PATH)
orders.head()

# 字段含义
- id ：主键
- data_source ：'数据来源：1 驾安配订单, 2 驾安配询价单, 3 驾安配报价单',      
- data_source_idx ：'数据来源唯一标识',           
- data_vehicletype_name ：'原始车型名称',           
- data_vehicletype_name_split ：'原始车型名称分词',           
- vehicle_code ：'精友车型编码',           
- brand_name ：'通用品牌',           
- manufacturer ：'主机厂品牌（车厂）',           
- category_name ：'车系',           
- category_short_name ：'车系简写名称',           
- year_pattern ：'年款',           
- vehicle_pattern ：'车型款型',           
- announcement_no ：'公告号',           
- engine ：'排量',           
- maingearbox ：'变速箱类型',           
- gearbox ：'变速箱',           
- vehicle_type ：'车辆类型',           
- part_name ：'配件名称',           
- oe ：'OE',           
- regularize_oe ：'OE（去除符号）',           
- part_jy_name ：'精友配件标准名称',           
- part_group ：'标准配件名称分组',           
- province_code ：'所在省代码',           
- province_name ：'所在省名称',           
- city_code ：'所在市代码',           
- city_name ：'所在市名称',           
- part_price ：成交价
- part_price_type_code ：'价格类型：01 成交价；02 供应商报价',         
- part_type_code ：'配件品质代码：01 原厂件;02 品牌件; 3 配套件; 4 流通原厂件;05 配套品牌件;06 售后品牌件;07 经济适用件;08 再制造件;09 拆车回用件',
- order_channel_source ：'渠道来源：JANP-驾安配;PICC-人保财险;ZKWG-中科万国;STEN-偲腾',           
- create_time ：创建时间



In [ ]:
# 查看数据信息
# orders.info()

In [ ]:
data = orders[['id','data_vehicletype_name','brand_name','part_name','oe','regularize_oe','part_jy_name','part_price','part_type_code','order_channel_source','province_name','city_name','data_source_idx','category_name','part_group','vehicle_type','part_price_type_code','data_source','province_name']]


In [ ]:
# data.info()

In [ ]:
# data.head()

In [ ]:
data_oe = data[(data['regularize_oe'].notnull())&(data['regularize_oe'].str.len()>3)&(data['part_type_code'].isin([1])&(data['part_price_type_code']==1))]
data_oe = data_oe.loc[~(data_oe['regularize_oe'].str.contains('JYJY'))]

In [ ]:
a = data_oe['regularize_oe'].value_counts()
# print(a) # pandas.core.series.Series
for k in range(100):
   print(a.index[k], a[k]) 

In [ ]:
data_oe_sample = data_oe[(data_oe['regularize_oe']=='3732020C0100')]
data_oe_sample = data_oe_sample[['id','regularize_oe', 'part_name', 'part_price','province_name']]
# data_oe_sample.sort_values(by='part_price')
# data_oe_sample['part_price'].hist()

In [ ]:
data_oe_sample.describe()

In [ ]:
# 均值+-标准差，作为上下线
import math
def cleanData_range(data, col):
    f_data_is_ok = False
    f_mean = data[col].mean()
    f_median = data[col].median()
    f_std = data[col].std()
    b_temp = math.isnan(f_mean) | math.isnan(f_std) | (f_mean == 0.0) | (f_std==0.0) | len(data)<=2
    # print("b_temp is ",b_temp)
    if b_temp:
        return data, True
    if f_std / f_mean > PARAM_TIMES:
        f_std = f_mean * PARAM_TIMES
    elif f_std / f_mean <= MIN_PARAM_TIMES:
        f_data_is_ok = True
    f_range_t = f_mean + f_std
    f_range_b = f_mean - f_std
    data_return = data[(data[col]>=f_range_b)&(data[col]<=f_range_t)]
    return data_return, f_data_is_ok


In [ ]:
# 箱体图算法：四分位Q3+(Q3-Q1)*1.5 > x > Q1-(Q3-Q1)*1.5 为异常值
import math
def cleanData_4p(data, col):
    f_data_is_ok = False
    f_q1 = data[col].quantile(0.25)
    f_q3 = data[col].quantile(0.75)
    delta_q31 = f_q3 - f_q1
    # 上下边缘
    b_edge = f_q1 - 1.5 * delta_q31
    t_edge = f_q3 + 1.5 * delta_q31
    b_temp = math.isnan(delta_q31) | math.isnan(f_q3) | (delta_q31 == 0.0) | (f_q3==0.0) # | len(data)<=2
    # print("b_temp is ",b_temp)
    if b_temp:
        return data, True
    data_return = data[(data[col]<=t_edge)&(data[col]>=b_edge)]
    if (len(data_return) == len(data)):
        f_data_is_ok = True
    return data_return, f_data_is_ok


In [ ]:
continue_flag = True
i = 1
print("================================================================")
print("OE:", data_oe_sample.iloc[0]['regularize_oe'])
while continue_flag:
    data_oe_sample, result = cleanData_range(data_oe_sample, 'part_price')
    print("第",i,"轮循环结果：", result)
    continue_flag = not result
    print(data_oe_sample.describe())
    print("-------------------------------------------\n")
    i+=1

In [ ]:
data_oe_sample.sort_values(by="part_price")

In [ ]:
data_oe_sample = data_oe[(data_oe['regularize_oe']=='3732020C0100')]
data_oe_sample = data_oe_sample[['id','regularize_oe', 'part_name', 'part_price','province_name']]
continue_flag = True
i = 1
print("================================================================")
print("OE:", data_oe_sample.iloc[0]['regularize_oe'])
while continue_flag:
    data_oe_sample, result = cleanData_4p(data_oe_sample, 'part_price')
    print("第",i,"轮循环结果：", result)
    continue_flag = not result
    print(data_oe_sample.describe())
    print("-------------------------------------------\n")
    i+=1

In [ ]:
data_oe_sample.sort_values(by="part_price")